<a href="https://colab.research.google.com/github/harperd17/energy_volatility_prediction/blob/main/notebooks/Modeling/Dashboard_Adam_of_Prod_Sklearn_withlogger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The goal of this notebook is to determine which model returns the highest R2 by feature reduction techniques and hp tuning

## This will set the prefix of your log file name

In [ ]:
name = "Adam"
#Not grid "adamgoldsteindatascience/energyprice-volatility-grid"

neptune_project_name = "adamgoldsteindatascience/energyprice-volatility"

#Overall Reproduceability
random_state = 0

In [ ]:
# Clones the main branch githubt to import: 
# 1.from fengine import Fengine
# 2.from sklearnlogger import SklearnLogger
!git clone https://github.com/harperd17/energy_volatility_prediction.git

fatal: destination path 'energy_volatility_prediction' already exists and is not an empty directory.


In [ ]:
!pip install neptune-client --quiet
import neptune.new as neptune

In [ ]:
import pandas as pd
import numpy as np
import sys
sys.path.append("energy_volatility_prediction/helpers/")

from sklearn.model_selection import train_test_split as tts
from sklearn.model_selection import cross_validate

import matplotlib.pyplot as plt

from fengine import Fengine
from sklearnlogger import SklearnLogger
import warnings

from google.colab import drive

drive.mount('./drive/')

%cd drive/Shareddrives/Data606_Energy

Drive already mounted at ./drive/; to attempt to forcibly remount, call drive.mount("./drive/", force_remount=True).
/content/drive/Shareddrives/Data606_Energy


# Logger and Plotting Functions

In [ ]:
import json
def get_neptune_key(username):

    with open("creds/creds.json","r") as f:

        api_token = json.load(f)["Users"][username]["neptune_key"]

    return api_token





# Analyze Results

In [ ]:
# Fetch project 'jack/sandbox'
project = neptune.get_project(name=neptune_project_name,api_token=get_neptune_key(name))

# Fetch all Runs metadata as Pandas DataFrame
runs_table_df = project.fetch_runs_table().to_pandas()

runs_table_df.columns =  [col.split("/")[2] if "Parameters" in col else col.split("/")[1] for col in runs_table_df.columns]

In [ ]:
runs_table_df.sort_values("Adj_R2_test",ascending=False)["Cust_Columns"][5]

"[['index', 'Year', 'State', 'Months Above Average Price Movement BZ=F', 'Months Above Average Price Volume BZ=F', 'Months Above Average Price Movement NG=F', 'Months Above Average Price Volume NG=F', 'Months Above Average Price Movement CL=F', 'Months Above Average Price Volume CL=F', 'Months Above Average Price Movement HO=F', 'Months Above Average Price Volume HO=F', 'Hot Summer Months', 'Cold Winter Months', 'Dry Summer Months', 'COV', 'target_COV', 'COM_percapita_mean_kWh_electricacc', 'RES_percapita_mean_kWh_electricacc', 'COM_percapita_mean_kWh_statepop', 'RES_percapita_mean_kWh_statepop', 'Agg_total_plant_MWh_x', 'Agg_total_plant_count_x', 'R_AB_MWh', 'R_AB_count', 'R_BFG_MWh', 'R_BFG_count', 'R_BIT_MWh', 'R_BIT_count', 'R_BLQ_MWh', 'R_BLQ_count', 'R_DFO_MWh', 'R_DFO_count', 'R_GEO_MWh', 'R_GEO_count', 'R_JF_MWh', 'R_JF_count', 'R_KER_MWh', 'R_KER_count', 'R_LFG_MWh', 'R_LFG_count', 'R_LIG_MWh', 'R_LIG_count', 'R_MSB_MWh', 'R_MSB_count', 'R_MSN_MWh', 'R_MSN_count', 'R_MSW_MWh',

In [ ]:
runs_table_df['creation_time'] = pd.to_datetime(runs_table_df['creation_time'])

runs_table_df['Cust_Model'] = runs_table_df['Cust_Model'].apply(lambda model: "XGBRegressor" if model == "type" else model)

In [ ]:
runs_table_df.query('Adj_R2_test > 0.40').sort_values("Adj_R2_test",ascending=False).head()

,creation_time,description,failed,hostname,id,modification_time,monitoring_time,name,owner,ping_time,running_time,size,state,tags,Adj_R2_test,Adj_R2_train,Coef_train,Cust_Columns,Cust_Model,Cust_Pipeline_funcs,Explained_Variance_Score_test,Explained_Variance_Score_train,MSE_test,MSE_train,alpha,alpha_1,alpha_2,alpha_init,base_estimator,base_estimator__alpha,base_estimator__alpha_1,base_estimator__alpha_2,base_estimator__alpha_init,base_estimator__alpha_per_target,base_estimator__alphas,base_estimator__base_estimator,base_estimator__base_estimator__base_estimator,base_estimator__base_estimator__base_estimator__categorical_features,base_estimator__base_estimator__base_estimator__early_stopping,base_estimator__base_estimator__base_estimator__l2_regularization,...,min_weight_fraction_leaf,missing,monotonic_cst,n_estimators,n_iter,n_iter_no_change,n_jobs,n_nonzero_coefs,n_subsamples,normalize,nthread,objective,oob_score,positive,precompute,random_state,reg_alpha,reg_lambda,scale_pos_weight,scoring,seed,selection,silent,solver,subsample,threshold_lambda,tol,validation_fraction,verbose,verbosity,warm_start,R2_test,R2_train,y_preds_test,y_preds_train,cpu,gpu_memory,memory,stderr,stdout
19,2021-12-01 22:20:08.350000+00:00,,False,94a359147139,EN-1683,2021-12-01 22:20:09.360000+00:00,600,Untitled,adamgoldsteindatascience,2021-12-01 22:20:09.360000+00:00,1.010,16892.0,idle,,0.769082,0.812980,None,"[['index', 'Year', 'State', 'Months Above Aver...",AdaBoostRegressor,"[[['scale_month_columns', None], ['drop_column...",NaN,NaN,0.000176,0.000141,NaN,NaN,NaN,NaN,HistGradientBoostingRegressor(learning_rate=0....,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.774601,0.817450,[0.01151657 0.02170265 0.0462405 0.02251547 0...,[0.03318682 0.04184224 0.05325788 0.03969497 0...,54.1,NaN,1.222168,NaN,"b'Shutting down background jobs, please wait a..."
0,2021-12-01 23:03:53.033000+00:00,,False,94a359147139,EN-1702,2021-12-01 23:03:54.023000+00:00,600,Untitled,adamgoldsteindatascience,2021-12-01 23:03:54.023000+00:00,0.989,16703.0,idle,,0.768890,0.769240,[0.41791302 0.08612174 0.01242776 0.02825161 0...,"[['index', 'Year', 'State', 'Months Above Aver...",XGBRegressor,"[[['scale_month_columns', None], ['drop_column...",NaN,NaN,0.000176,0.000174,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,None,NaN,500.0,NaN,NaN,1,NaN,NaN,NaN,None,reg:linear,NaN,NaN,NaN,0,0.0,1.0,1.0,NaN,None,NaN,None,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,0.774413,0.774755,[0.01164189 0.02066103 0.04423982 0.02363828 0...,[0.03153768 0.04303721 0.05009851 0.04252222 0...,1.5,NaN,1.221241,NaN,"b'Shutting down background jobs, please wait a..."
2,2021-12-01 22:41:40.726000+00:00,,False,94a359147139,EN-1700,2021-12-01 22:41:41.719000+00:00,600,Untitled,adamgoldsteindatascience,2021-12-01 22:41:41.719000+00:00,0.992,16304.0,idle,,0.767606,0.812027,None,"[['index', 'Year', 'State', 'Months Above Aver...",HistGradientBoostingRegressor,"[[['scale_month_columns', None], ['drop_column...",NaN,NaN,0.000177,0.000142,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,None,NaN,NaN,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,loss,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e-07,0.1,0,NaN,False,0.773160,0.816519,[0.0101247 0.02004124 0.0464448 0.02185191 0...,[0.03524172 0.041013 0.05224845 0.04109402 0...,2.3,NaN,1.221451,NaN,"b'Shutting down background jobs, please wait a..."
8,2021-12-01 22:40:02.029000+00:00,,False,94a359147139,EN-1694,2021-12-01 22:40:02.945000+00:00,600,Untitled,adamgoldsteindatascience,2021-12-01 22:40:02.945000+00:00,0.915,16313.0,idle,,0.767604,0.812035,None,"[['index', 'Year', 'State', 'Months Above Aver...",HistGradientBoostingRegressor,"[[['scale_month_columns', None], ['drop_column...",NaN,NaN,0.000177,0.000142,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,None,NaN,NaN,10,

In [ ]:


runs_table_df["Adj_R2_test_train_diff"] = runs_table_df["Adj_R2_test"] - runs_table_df["Adj_R2_train"]

In [ ]:
runs_table_df

,creation_time,description,failed,hostname,id,modification_time,monitoring_time,name,owner,ping_time,running_time,size,state,tags,Adj_R2_test,Adj_R2_train,Coef_train,Cust_Columns,Cust_Model,Cust_Pipeline_funcs,Explained_Variance_Score_test,Explained_Variance_Score_train,MSE_test,MSE_train,alpha,alpha_1,alpha_2,alpha_init,base_estimator,base_estimator__alpha,base_estimator__alpha_1,base_estimator__alpha_2,base_estimator__alpha_init,base_estimator__alpha_per_target,base_estimator__alphas,base_estimator__base_estimator,base_estimator__base_estimator__base_estimator,base_estimator__base_estimator__base_estimator__categorical_features,base_estimator__base_estimator__base_estimator__early_stopping,base_estimator__base_estimator__base_estimator__l2_regularization,...,missing,monotonic_cst,n_estimators,n_iter,n_iter_no_change,n_jobs,n_nonzero_coefs,n_subsamples,normalize,nthread,objective,oob_score,positive,precompute,random_state,reg_alpha,reg_lambda,scale_pos_weight,scoring,seed,selection,silent,solver,subsample,threshold_lambda,tol,validation_fraction,verbose,verbosity,warm_start,R2_test,R2_train,y_preds_test,y_preds_train,cpu,gpu_memory,memory,stderr,stdout,Adj_R2_test_train_diff
0,2021-12-01 23:03:53.033000+00:00,,False,94a359147139,EN-1702,2021-12-01 23:03:54.023000+00:00,600,Untitled,adamgoldsteindatascience,2021-12-01 23:03:54.023000+00:00,0.989,16703.0,idle,,0.768890,0.769240,[0.41791302 0.08612174 0.01242776 0.02825161 0...,"[['index', 'Year', 'State', 'Months Above Aver...",XGBRegressor,"[[['scale_month_columns', None], ['drop_column...",NaN,NaN,0.000176,0.000174,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,NaN,500.0,NaN,NaN,1,NaN,NaN,NaN,None,reg:linear,NaN,NaN,NaN,0,0.0,1.0,1.0,NaN,None,NaN,None,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,0.774413,0.774755,[0.01164189 0.02066103 0.04423982 0.02363828 0...,[0.03153768 0.04303721 0.05009851 0.04252222 0...,1.5,NaN,1.221241,NaN,"b'Shutting down background jobs, please wait a...",-0.000350
1,2021-12-01 22:41:52.160000+00:00,,False,94a359147139,EN-1701,2021-12-01 22:41:53.081000+00:00,600,Untitled,adamgoldsteindatascience,2021-12-01 22:41:53.081000+00:00,0.920,16311.0,idle,,0.765509,0.778731,None,"[['index', 'Year', 'State', 'Months Above Aver...",HistGradientBoostingRegressor,"[[['scale_month_columns', None], ['drop_column...",NaN,NaN,0.000179,0.000167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,None,NaN,NaN,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,loss,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e-07,0.1,0,NaN,False,0.771113,0.784019,[0.01088393 0.02118624 0.04453016 0.02272137 0...,[0.03275437 0.04331138 0.05246633 0.04117258 0...,4.2,NaN,1.221981,NaN,"b'Shutting down background jobs, please wait a...",-0.013222
2,2021-12-01 22:41:40.726000+00:00,,False,94a359147139,EN-1700,2021-12-01 22:41:41.719000+00:00,600,Untitled,adamgoldsteindatascience,2021-12-01 22:41:41.719000+00:00,0.992,16304.0,idle,,0.767606,0.812027,None,"[['index', 'Year', 'State', 'Months Above Aver...",HistGradientBoostingRegressor,"[[['scale_month_columns', None], ['drop_column...",NaN,NaN,0.000177,0.000142,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,None,NaN,NaN,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,loss,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e-07,0.1,0,NaN,False,0.773160,0.816519,[0.0101247 0.02004124 0.0464448 0.02185191 0...,[0.03524172 0.041013 0.05224845 0.04109402 0...,2.3,NaN,1.221451,NaN,"b'Shutting down background jobs, please wait a...",-0.044421
3,2021-12-01 22:40:52.267000+00:00,,False,94a359147139,EN-1699,2021-12-01 22:40:53.225000+00:00,600,Untitled,adamgoldsteindatascience,2021-12-01 22:40:53.225000+00:00,0.958,16312.0,idle,,0.758256,0.804831,None,"[['index', 'Year', 'State', 'Months Above Aver...",HistGradientBoostingRegressor,"[[['scale_month_columns', None], ['drop_column...",NaN,NaN,0.000184,0.000147,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,None,NaN,NaN,10,NaN,NaN,NaN,NaN

In [ ]:

df_max = runs_table_df[["Cust_Model","base_estimator","Adj_R2_test","Adj_R2_train","MSE_test","Adj_R2_test_train_diff","n_estimators","learning_rate","max_depth","Cust_Columns","Cust_Pipeline_funcs","Coef_train"]].reset_index()

In [ ]:
#Unique Experiments

df_max["Cust_Columns"].unique().shape

(15,)

In [ ]:
vis = df_max.query('Adj_R2_test >= 0.60').query('Adj_R2_test_train_diff < 0.03').query('Adj_R2_train < 0.79').drop(columns="Adj_R2_test_train_diff").sort_values("Adj_R2_test",ascending=False).drop_duplicates(["Cust_Model","n_estimators","learning_rate","max_depth"])

In [ ]:
def highlight_max(x, color):
    return np.where(x == np.nanmax(x.to_numpy()), f"background-color:{color};", None)

In [ ]:
pd.set_option('max_colwidth', 50)

a = vis[["Cust_Model","base_estimator","Adj_R2_test"]].groupby("Cust_Model").max().sort_values("Adj_R2_test",ascending=False)


a["base_estimator"] = a["base_estimator"].apply(lambda x: x if x == np.nan else str(x).split("(")[0])

a.style.apply(highlight_max, color='lightgreen',subset=["Adj_R2_test"])



,base_estimator,Adj_R2_test
Cust_Model,,
XGBRegressor,nan,0.768890
HistGradientBoostingRegressor,nan,0.766902
BaggingRegressor,HistGradientBoostingRegressor,0.763546
AdaBoostRegressor,LassoLarsIC,0.763318
ExtraTreesRegressor,nan,0.713314
GradientBoostingRegressor,nan,0.695125
RandomForestRegressor,nan,0.685285


In [ ]:
pd.set_option('max_colwidth', 0)



a = vis[["Cust_Model","base_estimator","Adj_R2_test","Cust_Columns","Coef_train"]].groupby("Cust_Model").max().sort_values("Adj_R2_test",ascending=False)

a["col_len"] = a["Cust_Columns"].apply(lambda x: len(x[0]))
a["coef_len"] = a["Coef_train"].apply(lambda x: len(x))

In [ ]:
unique_experiments

,Cust_Model,base_estimator,Adj_R2_test,Adj_R2_train,MSE_test,Adj_R2_test_train_diff,n_estimators,learning_rate,max_depth,Cust_Columns,Cust_Pipeline_funcs,Coef_train
0,XGBRegressor,NaN,0.768890,0.769240,0.000176,-0.000350,500.0,0.100000,1,"[['index', 'Year', 'State', 'Months Above Average Price Movement BZ=F', 'Months Above Average Price Volume BZ=F', 'Months Above Average Price Movement NG=F', 'Months Above Average Price Volume NG=F', 'Months Above Average Price Movement CL=F', 'Months Above Average Price Volume CL=F', 'Months Above Average Price Movement HO=F', 'Months Above Average Price Volume HO=F', 'Hot Summer Months', 'Cold Winter Months', 'Dry Summer Months', 'COV', 'target_COV', 'COM_percapita_mean_kWh_electricacc', 'RES_percapita_mean_kWh_electricacc', 'COM_percapita_mean_kWh_statepop', 'RES_percapita_mean_kWh_statepop', 'Agg_total_plant_MWh_x', 'Agg_total_plant_count_x', 'R_AB_MWh', 'R_AB_count', 'R_BFG_MWh', 'R_BFG_count', 'R_BIT_MWh', 'R_BIT_count', 'R_BLQ_MWh', 'R_BLQ_count', 'R_DFO_MWh', 'R_DFO_count', 'R_GEO_MWh', 'R_GEO_count', 'R_JF_MWh', 'R_JF_count', 'R_KER_MWh', 'R_KER_count', 'R_LFG_MWh', 'R_LFG_count', 'R_LIG_MWh', 'R_LIG_count', 'R_MSB_MWh', 'R_MSB_count', 'R_MSN_MWh', 'R_MSN_count', 'R_MSW_MWh', 'R_MSW_count', 'R_MWH_MWh', 'R_MWH_count', 'R_NG_MWh', 'R_NG_count', 'R_NUC_MWh', 'R_NUC_count', 'R_OBG_MWh', 'R_OBG_count', 'R_OBS_MWh', 'R_OBS_count', 'R_OG_MWh', 'R_OG_count', 'R_OTH_MWh', 'R_OTH_count', 'R_PC_MWh', 'R_PC_count', 'R_PG_MWh', 'R_PG_count', 'R_PUR_MWh', 'R_PUR_count', 'R_RC_MWh', 'R_RC_count', 'R_RFO_MWh', 'R_RFO_count', 'R_SC_MWh', 'R_SC_count', 'R_SGC_MWh', 'R_SGC_count', 'R_SLW_MWh', 'R_SLW_count', 'R_SUB_MWh', 'R_SUB_count', 'R_SUN_MWh', 'R_SUN_count', 'R_TDF_MWh', 'R_TDF_count', 'R_WAT_MWh', 'R_WAT_count', 'R_WC_MWh', 'R_WC_count', 'R_WDS_MWh', 'R_WDS_count', 'R_WH_MWh', 'R_WH_count', 'R_WND_MWh', 'R_WND_count', 'R_WO_MWh', 'R_WO_count', 'Agg_R_Biomass_MWh', 'Agg_R_Biomass_count', 'Agg_R_Coal_MWh', 'Agg_R_Coal_count', 'Agg_R_Geothermal_MWh', 'Agg_R_Geothermal_count', 'Agg_R_Hydroelectric_MWh', 'Agg_R_Hydroelectric_count', 'Agg_R_Municipal_Waste_MWh', 'Agg_R_Municipal_Waste_count', 'Agg_R_Natural_Gas_MWh', 'Agg_R_Natural_Gas_count', 'Agg_R_Nuclear_MWh', 'Agg_R_Nuclear_count', 'Agg_R_Oil_Gas_MWh', 'Agg_R_Oil_Gas_count', 'Agg_R_Other_MWh', 'Agg_R_Other_count', 'Agg_R_Solar_MWh', 'Agg_R_Solar_count', 'Agg_R_Wind_MWh', 'Agg_R_Wind_count', 'Agg_R_batteries_or_other_use_of_electricity_as_an_energy_source_MWh', 'Agg_R_batteries_or_other_use_of_electricity_as_an_energy_source_count', 'Agg_total_plant_MWh_y', 'Agg_total_plant_count_y', 'year.1']]","[[['scale_month_columns', None], ['drop_columns', ['Year', 'year.1']], ['treat_bad_values', None], ('outlier_thresh', 3), ('col_not_startswith', 'R_'), ('col_not_startswith', 'Agg_R'), ('col_not_endswith', '_y')]]",[0.41791302 0.08612174 0.01242776 0.02825161 0.02143938 0.\n 0.00700625 0.00959999 0.02702856 0.0200533 0.00581729 0.\n 0.29760107 0.00956087 0.0138461 0.00714015 0.01001407 0.01490072\n 0.01127814]
1,HistGradientBoostingRegressor,NaN,0.765509,0.778731,0.000179,-0.013222,NaN,0.060000,2,"[['index', 'Year', 'State', 'Months Above Average Price Movement BZ=F', 'Months Above Average Price Volume BZ=F', 'Months Above Average Price Movement NG=F', 'Months Above Average Price Volume NG=F', 'Months Above Average Price Movement CL=F', 'Months Above Average Price Volume CL=F', 'Months Above Average Price Movement HO=F', 'Months Above Average Price Volume HO=F', 'Hot Summer Months', 'Cold Winter Months', 'Dry Summer Months', 'COV', 'target_COV', 'COM_percapita_mean_kWh_electricacc', 'RES_percapita_mean_kWh_electricacc', 'COM_percapita_mean_kWh_statepop', 'RES_percapita_mean_kWh_statepop', 'Agg_total_plant_MWh_x', 'Agg_total_plant_count_x', 'R_AB_MWh', 'R_AB_count', 'R_BFG_MWh', 'R_BFG_count', 'R_BIT_MWh', 'R_BIT_count', 'R_BLQ_MWh', 'R_BLQ_count', 'R_DFO_MWh', 'R_DFO_count', 'R_GEO_MWh', 'R_GEO_count', 'R_JF_MWh', 'R_JF_count', 'R_KER_MWh', 'R_KER_count',

In [ ]:
unique_experiments

,index,Cust_Model,base_estimator,Adj_R2_test,Adj_R2_train,MSE_test,Adj_R2_test_train_diff,n_estimators,learning_rate,max_depth,Cust_Columns,Cust_Pipeline_funcs,Coef_train
0,0,XGBRegressor,NaN,0.768890,0.769240,0.000176,-0.000350,500.0,0.100000,1,"[['index', 'Year', 'State', 'Months Above Average Price Movement BZ=F', 'Months Above Average Price Volume BZ=F', 'Months Above Average Price Movement NG=F', 'Months Above Average Price Volume NG=F', 'Months Above Average Price Movement CL=F', 'Months Above Average Price Volume CL=F', 'Months Above Average Price Movement HO=F', 'Months Above Average Price Volume HO=F', 'Hot Summer Months', 'Cold Winter Months', 'Dry Summer Months', 'COV', 'target_COV', 'COM_percapita_mean_kWh_electricacc', 'RES_percapita_mean_kWh_electricacc', 'COM_percapita_mean_kWh_statepop', 'RES_percapita_mean_kWh_statepop', 'Agg_total_plant_MWh_x', 'Agg_total_plant_count_x', 'R_AB_MWh', 'R_AB_count', 'R_BFG_MWh', 'R_BFG_count', 'R_BIT_MWh', 'R_BIT_count', 'R_BLQ_MWh', 'R_BLQ_count', 'R_DFO_MWh', 'R_DFO_count', 'R_GEO_MWh', 'R_GEO_count', 'R_JF_MWh', 'R_JF_count', 'R_KER_MWh', 'R_KER_count', 'R_LFG_MWh', 'R_LFG_count', 'R_LIG_MWh', 'R_LIG_count', 'R_MSB_MWh', 'R_MSB_count', 'R_MSN_MWh', 'R_MSN_count', 'R_MSW_MWh', 'R_MSW_count', 'R_MWH_MWh', 'R_MWH_count', 'R_NG_MWh', 'R_NG_count', 'R_NUC_MWh', 'R_NUC_count', 'R_OBG_MWh', 'R_OBG_count', 'R_OBS_MWh', 'R_OBS_count', 'R_OG_MWh', 'R_OG_count', 'R_OTH_MWh', 'R_OTH_count', 'R_PC_MWh', 'R_PC_count', 'R_PG_MWh', 'R_PG_count', 'R_PUR_MWh', 'R_PUR_count', 'R_RC_MWh', 'R_RC_count', 'R_RFO_MWh', 'R_RFO_count', 'R_SC_MWh', 'R_SC_count', 'R_SGC_MWh', 'R_SGC_count', 'R_SLW_MWh', 'R_SLW_count', 'R_SUB_MWh', 'R_SUB_count', 'R_SUN_MWh', 'R_SUN_count', 'R_TDF_MWh', 'R_TDF_count', 'R_WAT_MWh', 'R_WAT_count', 'R_WC_MWh', 'R_WC_count', 'R_WDS_MWh', 'R_WDS_count', 'R_WH_MWh', 'R_WH_count', 'R_WND_MWh', 'R_WND_count', 'R_WO_MWh', 'R_WO_count', 'Agg_R_Biomass_MWh', 'Agg_R_Biomass_count', 'Agg_R_Coal_MWh', 'Agg_R_Coal_count', 'Agg_R_Geothermal_MWh', 'Agg_R_Geothermal_count', 'Agg_R_Hydroelectric_MWh', 'Agg_R_Hydroelectric_count', 'Agg_R_Municipal_Waste_MWh', 'Agg_R_Municipal_Waste_count', 'Agg_R_Natural_Gas_MWh', 'Agg_R_Natural_Gas_count', 'Agg_R_Nuclear_MWh', 'Agg_R_Nuclear_count', 'Agg_R_Oil_Gas_MWh', 'Agg_R_Oil_Gas_count', 'Agg_R_Other_MWh', 'Agg_R_Other_count', 'Agg_R_Solar_MWh', 'Agg_R_Solar_count', 'Agg_R_Wind_MWh', 'Agg_R_Wind_count', 'Agg_R_batteries_or_other_use_of_electricity_as_an_energy_source_MWh', 'Agg_R_batteries_or_other_use_of_electricity_as_an_energy_source_count', 'Agg_total_plant_MWh_y', 'Agg_total_plant_count_y', 'year.1']]","[[['scale_month_columns', None], ['drop_columns', ['Year', 'year.1']], ['treat_bad_values', None], ('outlier_thresh', 3), ('col_not_startswith', 'R_'), ('col_not_startswith', 'Agg_R'), ('col_not_endswith', '_y')]]",[0.41791302 0.08612174 0.01242776 0.02825161 0.02143938 0.\n 0.00700625 0.00959999 0.02702856 0.0200533 0.00581729 0.\n 0.29760107 0.00956087 0.0138461 0.00714015 0.01001407 0.01490072\n 0.01127814]
1,1,HistGradientBoostingRegressor,NaN,0.765509,0.778731,0.000179,-0.013222,NaN,0.060000,2,"[['index', 'Year', 'State', 'Months Above Average Price Movement BZ=F', 'Months Above Average Price Volume BZ=F', 'Months Above Average Price Movement NG=F', 'Months Above Average Price Volume NG=F', 'Months Above Average Price Movement CL=F', 'Months Above Average Price Volume CL=F', 'Months Above Average Price Movement HO=F', 'Months Above Average Price Volume HO=F', 'Hot Summer Months', 'Cold Winter Months', 'Dry Summer Months', 'COV', 'target_COV', 'COM_percapita_mean_kWh_electricacc', 'RES_percapita_mean_kWh_electricacc', 'COM_percapita_mean_kWh_statepop', 'RES_percapita_mean_kWh_statepop', 'Agg_total_plant_MWh_x', 'Agg_total_plant_count_x', 'R_AB_MWh', 'R_AB_count', 'R_BFG_MWh', 'R_BFG_count', 'R_BIT_MWh', 'R_BIT_count', 'R_BLQ_MWh', 'R_BLQ_count', 'R_DFO_MWh', 'R_DFO_count', 'R_GEO_MWh', 'R_GEO_count', 'R_JF_MWh', 'R_JF_count', 'R_KER_MWh', 'R_K

In [ ]:
#Unique Experiments

unique_experiments = df_max.drop_duplicates(["Cust_Model","n_estimators","learning_rate","max_depth"]).reset_index()

unique_experiments

,index,Cust_Model,base_estimator,Adj_R2_test,Adj_R2_train,MSE_test,Adj_R2_test_train_diff,n_estimators,learning_rate,max_depth,Cust_Columns,Cust_Pipeline_funcs,Coef_train
0,0,XGBRegressor,NaN,0.768890,0.769240,0.000176,-0.000350,500.0,0.100000,1,"[['index', 'Year', 'State', 'Months Above Average Price Movement BZ=F', 'Months Above Average Price Volume BZ=F', 'Months Above Average Price Movement NG=F', 'Months Above Average Price Volume NG=F', 'Months Above Average Price Movement CL=F', 'Months Above Average Price Volume CL=F', 'Months Above Average Price Movement HO=F', 'Months Above Average Price Volume HO=F', 'Hot Summer Months', 'Cold Winter Months', 'Dry Summer Months', 'COV', 'target_COV', 'COM_percapita_mean_kWh_electricacc', 'RES_percapita_mean_kWh_electricacc', 'COM_percapita_mean_kWh_statepop', 'RES_percapita_mean_kWh_statepop', 'Agg_total_plant_MWh_x', 'Agg_total_plant_count_x', 'R_AB_MWh', 'R_AB_count', 'R_BFG_MWh', 'R_BFG_count', 'R_BIT_MWh', 'R_BIT_count', 'R_BLQ_MWh', 'R_BLQ_count', 'R_DFO_MWh', 'R_DFO_count', 'R_GEO_MWh', 'R_GEO_count', 'R_JF_MWh', 'R_JF_count', 'R_KER_MWh', 'R_KER_count', 'R_LFG_MWh', 'R_LFG_count', 'R_LIG_MWh', 'R_LIG_count', 'R_MSB_MWh', 'R_MSB_count', 'R_MSN_MWh', 'R_MSN_count', 'R_MSW_MWh', 'R_MSW_count', 'R_MWH_MWh', 'R_MWH_count', 'R_NG_MWh', 'R_NG_count', 'R_NUC_MWh', 'R_NUC_count', 'R_OBG_MWh', 'R_OBG_count', 'R_OBS_MWh', 'R_OBS_count', 'R_OG_MWh', 'R_OG_count', 'R_OTH_MWh', 'R_OTH_count', 'R_PC_MWh', 'R_PC_count', 'R_PG_MWh', 'R_PG_count', 'R_PUR_MWh', 'R_PUR_count', 'R_RC_MWh', 'R_RC_count', 'R_RFO_MWh', 'R_RFO_count', 'R_SC_MWh', 'R_SC_count', 'R_SGC_MWh', 'R_SGC_count', 'R_SLW_MWh', 'R_SLW_count', 'R_SUB_MWh', 'R_SUB_count', 'R_SUN_MWh', 'R_SUN_count', 'R_TDF_MWh', 'R_TDF_count', 'R_WAT_MWh', 'R_WAT_count', 'R_WC_MWh', 'R_WC_count', 'R_WDS_MWh', 'R_WDS_count', 'R_WH_MWh', 'R_WH_count', 'R_WND_MWh', 'R_WND_count', 'R_WO_MWh', 'R_WO_count', 'Agg_R_Biomass_MWh', 'Agg_R_Biomass_count', 'Agg_R_Coal_MWh', 'Agg_R_Coal_count', 'Agg_R_Geothermal_MWh', 'Agg_R_Geothermal_count', 'Agg_R_Hydroelectric_MWh', 'Agg_R_Hydroelectric_count', 'Agg_R_Municipal_Waste_MWh', 'Agg_R_Municipal_Waste_count', 'Agg_R_Natural_Gas_MWh', 'Agg_R_Natural_Gas_count', 'Agg_R_Nuclear_MWh', 'Agg_R_Nuclear_count', 'Agg_R_Oil_Gas_MWh', 'Agg_R_Oil_Gas_count', 'Agg_R_Other_MWh', 'Agg_R_Other_count', 'Agg_R_Solar_MWh', 'Agg_R_Solar_count', 'Agg_R_Wind_MWh', 'Agg_R_Wind_count', 'Agg_R_batteries_or_other_use_of_electricity_as_an_energy_source_MWh', 'Agg_R_batteries_or_other_use_of_electricity_as_an_energy_source_count', 'Agg_total_plant_MWh_y', 'Agg_total_plant_count_y', 'year.1']]","[[['scale_month_columns', None], ['drop_columns', ['Year', 'year.1']], ['treat_bad_values', None], ('outlier_thresh', 3), ('col_not_startswith', 'R_'), ('col_not_startswith', 'Agg_R'), ('col_not_endswith', '_y')]]",[0.41791302 0.08612174 0.01242776 0.02825161 0.02143938 0.\n 0.00700625 0.00959999 0.02702856 0.0200533 0.00581729 0.\n 0.29760107 0.00956087 0.0138461 0.00714015 0.01001407 0.01490072\n 0.01127814]
1,1,HistGradientBoostingRegressor,NaN,0.765509,0.778731,0.000179,-0.013222,NaN,0.060000,2,"[['index', 'Year', 'State', 'Months Above Average Price Movement BZ=F', 'Months Above Average Price Volume BZ=F', 'Months Above Average Price Movement NG=F', 'Months Above Average Price Volume NG=F', 'Months Above Average Price Movement CL=F', 'Months Above Average Price Volume CL=F', 'Months Above Average Price Movement HO=F', 'Months Above Average Price Volume HO=F', 'Hot Summer Months', 'Cold Winter Months', 'Dry Summer Months', 'COV', 'target_COV', 'COM_percapita_mean_kWh_electricacc', 'RES_percapita_mean_kWh_electricacc', 'COM_percapita_mean_kWh_statepop', 'RES_percapita_mean_kWh_statepop', 'Agg_total_plant_MWh_x', 'Agg_total_plant_count_x', 'R_AB_MWh', 'R_AB_count', 'R_BFG_MWh', 'R_BFG_count', 'R_BIT_MWh', 'R_BIT_count', 'R_BLQ_MWh', 'R_BLQ_count', 'R_DFO_MWh', 'R_DFO_count', 'R_GEO_MWh', 'R_GEO_count', 'R_JF_MWh', 'R_JF_count', 'R_KER_MWh', 'R_K

In [ ]:
# df=df_max.sort_values(by=['Adj_R2_test', 'Adj_R2_train'],inplace=True, ascending=False)
# #df.sort_values(by=['Brand'], inplace=True, ascending=False)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
import plotly.express as px


(px.line(runs_table_df.query('Adj_R2_test > 0.40').sort_values("creation_time",ascending=False),x=runs_table_df.index,y="Adj_R2_test",color="Cust_Model",hover_data=["R2_test","Adj_R2_test","MSE_test"],
        
        log_y=True,title="Model Performance - Adjusted R2")
        
        .for_each_trace(lambda t: t.update(name=t.name.replace("Cust_Model=","")))
        
        )

ValueError: ignored

In [ ]:
runs_table_df["creation_time"] = list(runs_table_df.index)



fig = (px.line(runs_table_df.query('Adj_R2_test > 0.40').sort_values("creation_time",ascending=False),x="creation_time",y="Adj_R2_test",color="Cust_Model",hover_data=["Adj_R2_test","Adj_R2_test"],
        
        log_y=True,title="Model Performance - Adj_R2")
        
        .for_each_trace(lambda t: t.update(name=t.name.replace("Cust_Model=","")))
        
        )


fig.update_layout(
    xaxis_title="Index",)

In [ ]:

(px.line(runs_table_df.query('Adj_R2_test > 0.40').sort_values("creation_time",ascending=False),x="creation_time",y="MSE_test",color="Cust_Model",hover_data=["Adj_R2_test","Adj_R2_test"],
        
        log_y=True,title="Model Performance - Adjusted R2")
        
        .for_each_trace(lambda t: t.update(name=t.name.replace("Cust_Model=","")))
        
        )